# Проект: вариант 4

Данный вариант итогового проекта является примером тестового задания из компании OZON. 

## Задание 1

В данном задании вам предлагается написать несколько SQL-запросов для определенных таблиц. В каждом случае вам НЕ будет предоставленна таблица в явном виде. Однако, вам будет дан пример таблицы с данными.

### 1. Посчитать % изменение количества клиентов, совершивших покупку, месяц-к-месяцу 

    SELECT
        order_month, 
        (quantity_of_clients - quantity_of_clients_for_previous_month)/CAST(quantity_of_clients_for_previous_month AS FLOAT)*100 AS precent_changing_clients_month_by_month
    FROM
        (
        SELECT *, 
            LAG(quantity_of_clients) OVER (ORDER BY order_month ASC) AS quantity_of_clients_for_previous_month 
        FROM  
            ( 
            SELECT
                    dateadd(day, - datepart(day, order_date) + 1, convert(date, order_date)) AS order_month,
                    COUNT(DISTINCT client_id) AS quantity_of_clients
            FROM base.table
            GROUP BY dateadd(day, - datepart(day, order_date) + 1, convert(date, order_date))
            ) AS tabl1 
        ) AS tabl2

### 2. Вывести сумму GMV (Gross Merchandise Value) с нарастающим итогом по дням 

SELECT

    fact_date,
    SUM(gmv) OVER (ORDER BY fact_date ASC) AS GMV_cumulative_total_by_day
FROM base.table

решение для clickhouse:

    SELECT
        fact_date,
        SUM(gmv) OVER w AS GMV_cumulative_total_by_day
    FROM base.table
    WINDOW w AS (ORDER BY fact_date
                ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW)
    ORDER BY fact_date

### 3. Получить время отклика на каждое письмо (письмо идентифицируется по полю mail_id), отправленное пользователем mr_employee@ozon.ru.

    SELECT 
        mail_id,  
        mail_subject,
        response_time_in_days
    FROM 
        (
        SELECT
            mail_id, mail_from, mail_to, mail_subject, timestamp, timestamp_next,
            datediff(d, timestamp_, timestamp_new) AS response_time_in_days,
            MAX(CASE WHEN mail_from ='mr_employee@ozon.ru' AND timestamp_next IS NOT NULL THEN 1 ELSE 0 END) FLAG
        FROM 
                (
                    SELECT
                        *,
                        Lead(timestamp) OVER (PARTITION BY mail_subject ORDER BY timestamp
        ASC) AS timestamp_next
                    FROM base.table
                ) tabl1
        GROUP by mail_id, mail_from, mail_to, mail_subject, timestamp, timestamp_next
        ) tabl2
    WHERE FLAG=1

### 4. Вывести id сотрудников с разницей в заработной плате в пределах 5000 рублей 

    SELECT
        employee_id AS id_employees_with_salary_difference_5000_rubles,
        employee_id_next AS id_employees_next_with_salary_difference_5000_rubles
    FROM 
        (
        SELECT 
            *,
            salary_rub_next_employee - salary_rub AS salary_difference,
            Lead(employee_id) OVER (ORDER BY salary_rub ASC) AS employee_id_next
        FROM 
            (
            SELECT 
                *,
                Lead(salary_rub) OVER (ORDER BY salary_rub ASC) AS salary_rub_next_employee
            FROM base.table
            ) AS tabl1
        ) AS tabl2
    WHERE salary_difference <5000

решение для clickhouse:

    SELECT
        employee_id,
        STRING_AGG(CAST(employee_id AS varchar), ', ') OVER w AS id_employees_with_salary_difference_5000_rubles
    FROM base.table
    WINDOW w AS (ORDER BY salary_rub
                RANGE BETWEEN 5000 preceding AND 5000 following)

## Задание 2 - Case Study. Опыт продавца

Схемы работы Маркетплейса

Маркетплейс OZON предлагает продавцам несколько схем для сотрудничества, среди них выделяют:

FBO (Fulfilment by OZON) – продавец размещает товары на складе маркетплейса. Маркетплейс сам отвечает за складскую обработку и логистику заказа до клиента. Доход площадки складывается из комиссии за витрину, складские операции и логистику.

FBS (Fulfilment by seller) – продавец самостоятельно хранит товары на своём складе и отвечает за складскую обработку. После получения заказа в Личном кабинете (ЛК), продавец его обрабатывает и передаёт в логистику Маркетплейса. Доход площадки складывается из комиссии за витрину и логистику.

FBS 2.0 / Real FBS / DBS (Delivery by seller) – продавец самостоятельно хранит товары на своём складе и отвечает за складскую обработку. После получения заказа в Личном кабинете (ЛК) продавец его обрабатывает и передаёт в любую удобную ему логистику. Доход площадки складывается из комиссии за витрину.

Личный кабинет продавца

Для начала своей работы продавец проходит регистрацию в личном кабинете продавца . Для получения первого заказа необходимо настроить параметры товаров, складов, логистики. Процесс занимает некоторое время, и продавец может уйти с площадки, так и не запустив продажи.

Задание к Case Study
1. Нарисовать процесс регистрации и выхода продавца на Маркетплейс от момента регистрации до 1-ой продажи по схеме DBS и описать основные отличия от схемы FBO и FBS;

2. Описать основные сложности, которые могут возникнуть на каждом этапе с момента регистрации и первой продажи по схеме DBS;

3. Разработать метрики, которые на ваш взгляд могут показать ключевые отличия в процессах между указанными выше схемами работы;

4. Установить нормативные значения (KPI) по метрикам и обосновать релевантность и реалистичность этих KPI

#### Смотрите, пожалуйста, презентацию Antipova_final_project_variant_4_task_2

## Задание 3

Метрики — показатели, с помощью которых измеряют эффективность процессов на каждом этапе.

Суть метрики - придумать такой численный показатель, который бы наиболее репрезентативно и одним числом показывал положение дел в конкретном процессе. Также, метрику должно быть удобно отслеживать в динамике, чтобы понимать, повлияли ли улучшения в бизнес-процессах положительно на какие-то аспекты бизнеса, или же отрицательно.

До расчета метрик нужно импортировать необходимые библиотеки 

In [1]:
import pandahouse as ph
import pandas as pd
import numpy as np
import datetime as dt

In [2]:
#объявляем параметры подключения
connection = {'host': 'http://clickhouse.beslan.pro:8080',
                      'database':'default',
                      'user':'student', 
                      'password':'dpo_python_2020'
                     }

In [3]:
#проверяем подключение тестовым запросом, предварительно посчитав кол-во строк в самом clickhouse 
query = """
SELECT
count(*)
FROM default.logistics_online
"""
df = ph.read_clickhouse(query, connection=connection)
df

,count()
0,6597


количество строк совпало с количеством строк в Clickhouse

Теперь нужно изучить сами данные. Возможно, не все типы корректно отображаются, либо есть какие-то пропуски, либо расхождения. Для проверки в Clickhouse используем функцию toTypeName() и проверяем типы данных

In [4]:
query = """
select
toTypeName(carrier_name) as carrier_name,
toTypeName(customer_cluster) as customer_cluster,
toTypeName(date_order) as date_order,
toTypeName(delivery_type) as delivery_type,
toTypeName(number_order) as number_order,
toTypeName(planned_data_order) as planned_data_order,
toTypeName(result_data_order) as result_data_order,
toTypeName(seller_ID) as seller_ID,
toTypeName(timeslot_number) as timeslot_number,
toTypeName(warehouse_cluster) as warehouse_cluster
from
default.logistics_online
limit 100
"""
df = ph.read_clickhouse(query, connection=connection)
df

,carrier_name,customer_cluster,date_order,delivery_type,number_order,planned_data_order,result_data_order,seller_ID,timeslot_number,warehouse_cluster
0,String,String,Nullable(String),String,Int64,Nullable(String),Nullable(String),Int64,Int8,String
1,String,String,Nullable(String),String,Int64,Nullable(String),Nullable(String),Int64,Int8,String
2,String,String,Nullable(String),String,Int64,Nullable(String),Nullable(String),Int64,Int8,String
3,String,String,Nullable(String),String,Int64,Nullable(String),Nullable(String),Int64,Int8,String
4,String,String,Nullable(String),String,Int64,Nullable(String),Nullable(String),Int64,Int8,String
...,...,...,...,...,...,...,...,...,...,...
95,String,String,Nullable(String),String,Int64,Nullable(String),Nullable(String),Int64,Int8,String
96,String,String,Nullable(String),String,Int64,Nullable(String),Nullable(String),Int64,Int8,String
97,String,String,Nullable(String),String,Int64,Nullable(String),Nullable(String),Int64,Int8,String
98,String,String,Nullable(String),String,Int64,Nullable(String),Nullable(String),Int64,Int8,String


В столбцах даты (дата оформления заказа, плановая дата доставки, фактическая дата доставки) прописаны в формате Nullable(String), их нужно конвертировать к виду Date с помощью функции toDate(), при конвертации NaN преобразовались в 1 января 1970 г., эту дату нужно будет отсечь в результирующем наборе.


проверяем полноту заполнения

In [5]:
query = """
select
count(carrier_name) as carrier_name,
count(customer_cluster) as customer_cluster,
count(date_order) as date_order,
count(delivery_type) as delivery_type,
count(number_order) as number_order,
count(planned_data_order) as planned_data_order,
count(result_data_order) as result_data_order,
count(seller_ID) as seller_ID,
count(timeslot_number) as timeslot_number,
count(warehouse_cluster) as warehouse_cluster
from
default.logistics_online
"""
df = ph.read_clickhouse(query, connection=connection)
df

,carrier_name,customer_cluster,date_order,delivery_type,number_order,planned_data_order,result_data_order,seller_ID,timeslot_number,warehouse_cluster
0,6597,6597,5972,6597,6597,6597,5707,6597,6597,6597


В date_order есть пустые поля, их нужно удалить при расчетах, так как дата оформления заказа - это одно из ключевых полей при анализе метрик в динамике. 

Также в данных встречаются случаи, когда дата оформления заказа больше даты доставки, такие данные следует пофиксить. 

Еще есть даты, когда дата плановой доставки больше даты доставки, такое возможно, но это также нужно учитывать при расчете некоторых метрик

проверяем оставшиеся две таблицы:

In [6]:
query = """
select
toTypeName(amount) as amount,
toTypeName(number_order) as number_order,
toTypeName(product_ID) as product_ID,
toTypeName(product_price) as product_price
from
default.product_online

limit 100
"""
df = ph.read_clickhouse(query, connection=connection)
df

,amount,number_order,product_ID,product_price
0,Int16,Int64,Int64,Float64
1,Int16,Int64,Int64,Float64
2,Int16,Int64,Int64,Float64
3,Int16,Int64,Int64,Float64
4,Int16,Int64,Int64,Float64
...,...,...,...,...
95,Int16,Int64,Int64,Float64
96,Int16,Int64,Int64,Float64
97,Int16,Int64,Int64,Float64
98,Int16,Int64,Int64,Float64


In [7]:
query = """
select
count(amount) as amount,
count(number_order) as number_order,
count(product_ID) as product_ID,
count(product_price) as product_price
from
default.product_online
"""
df = ph.read_clickhouse(query, connection=connection)
df

,amount,number_order,product_ID,product_price
0,12233,12233,12233,12233


In [8]:
query = """
select
toTypeName(amount_return) as amount_return,
toTypeName(number_order) as number_order,
toTypeName(product_ID) as product_ID,
toTypeName(return_date) as return_date
from
default.returns_online
"""
df = ph.read_clickhouse(query, connection=connection)
df

,amount_return,number_order,product_ID,return_date
0,Int8,Int64,Int64,Date
1,Int8,Int64,Int64,Date
2,Int8,Int64,Int64,Date
3,Int8,Int64,Int64,Date
4,Int8,Int64,Int64,Date
...,...,...,...,...
491,Int8,Int64,Int64,Date
492,Int8,Int64,Int64,Date
493,Int8,Int64,Int64,Date
494,Int8,Int64,Int64,Date


In [9]:
query = """
select
count(amount_return) as amount_return,
count(number_order) as number_order,
count(product_ID) as product_ID,
count(return_date) as return_date
from
default.returns_online
"""
df = ph.read_clickhouse(query, connection=connection)
df

,amount_return,number_order,product_ID,return_date
0,496,496,496,496


In [10]:
query = """
select
amount_return,
number_order,
product_ID,
return_date
from
default.returns_online
"""
df = ph.read_clickhouse(query, connection=connection)
df

,amount_return,number_order,product_ID,return_date
0,1,3031410942,983588011,1970-01-01
1,1,31014374140,983586554,2021-07-01
2,1,3147145173,983589156,2021-11-01
3,1,3141464669,983589362,2021-11-01
4,3,3131450699,983589729,2021-12-01
...,...,...,...,...
491,2,35714135014,983587402,1970-01-01
492,3,35714135014,983584567,1970-01-01
493,2,35714135014,983587402,1970-01-01
494,1,361501547,983581947,1970-01-01


### Задача 1.

Посчитать OnTime - Метрика своевременности доставки. (Доставили в плановую дату доставки или нет), в %

 - Необходимо видеть изменение метрики по дням;

 - Необходимы разрезы по перевозчикам, по типу доставки, по кластерам склада, по таймслотам;

In [11]:
query = """

SELECT
    ROUND(COUNT(CASE WHEN result_data_order == planned_data_order THEN number_order END)/COUNT(DISTINCT number_order)*100, 2) AS OnTime
FROM 
(
    SELECT
        toDateOrNull(concat(splitByString('-', assumeNotNull(date_order))[1], '-',
                            splitByString('-', assumeNotNull(date_order))[3], '-',
                            splitByString('-', assumeNotNull(date_order))[2])) as date_order,

        toDateOrNull(concat(splitByString('-', assumeNotNull(planned_data_order))[1], '-',
                            splitByString('-', assumeNotNull(planned_data_order))[3], '-',
                            splitByString('-', assumeNotNull(planned_data_order))[2])) as planned_data_order,

        toDateOrNull(concat(splitByString('-', assumeNotNull(result_data_order))[1], '-',
                            splitByString('-', assumeNotNull(result_data_order))[3], '-',
                            splitByString('-', assumeNotNull(result_data_order))[2])) as result_data_order,
        carrier_name, customer_cluster, delivery_type, number_order, seller_ID, warehouse_cluster, timeslot_number
    FROM default.logistics_online
    WHERE order_status = 'Доставлен' -- оставляем только фактически доставленные заказы
        AND date_order IS NOT NULL  -- избавляемся от данных без дат оформления заказов (так как такие данные невозможно анализирвать в динамике)
        AND date_order < result_data_order -- учитываем возможный баг 
        AND date_order < planned_data_order
        AND date_order != '1970-01-01'  -- учитываем конвертацию NaN в 1 января 1970
        AND planned_data_order != '1970-01-01'
        AND result_data_order != '1970-01-01'
) As tabl1

"""
task1 = ph.read_clickhouse(query, connection=connection)
task1

,OnTime
0,17.92


только 18% заказов доставляется в плановую дату доставки, но как выяснится ниже, есть заказы и их много, которые доставляются раньше оговоренного срока, что совсем неплохо

изменение метрики по дням:

    query = """

    SELECT
        date_order,
        ROUND(COUNT(CASE WHEN result_data_order == planned_data_order THEN number_order END)/COUNT(DISTINCT number_order)*100, 2) AS OnTime
    FROM 
    (
        SELECT
        toDateOrNull(concat(splitByString('-', assumeNotNull(date_order))[1], '-',
                            splitByString('-', assumeNotNull(date_order))[3], '-',
                            splitByString('-', assumeNotNull(date_order))[2])) as date_order,

        toDateOrNull(concat(splitByString('-', assumeNotNull(planned_data_order))[1], '-',
                            splitByString('-', assumeNotNull(planned_data_order))[3], '-',
                            splitByString('-', assumeNotNull(planned_data_order))[2])) as planned_data_order,

        toDateOrNull(concat(splitByString('-', assumeNotNull(result_data_order))[1], '-',
                            splitByString('-', assumeNotNull(result_data_order))[3], '-',
                            splitByString('-', assumeNotNull(result_data_order))[2])) as result_data_order,
        carrier_name, customer_cluster, delivery_type, number_order, seller_ID, warehouse_cluster, timeslot_number
        FROM default.logistics_online
        WHERE order_status = 'Доставлен' -- оставляем только фактически доставленные заказы
            AND date_order IS NOT NULL  -- избавляемся от данных без дат оформления заказов (так как такие данные невозможно анализирвать в динамике)
            AND date_order < result_data_order --учитываем возможный баг
            AND date_order < planned_data_order
            AND date_order != '1970-01-01'  -- учитываем конвертацию NaN в 1 января 1970
            AND planned_data_order != '1970-01-01'
            AND result_data_order != '1970-01-01'
    ) As tabl1
    GROUP BY date_order
    ORDER BY date_order ASC

    """
    task1 = ph.read_clickhouse(query, connection=connection)
    task1

так как система выдавала ошибку, связанную с форматом Nullable(Date), я сохранила результирующий набор в ClickHouse в csv

In [16]:
OnTime_by_days = pd.read_csv('OnTime_by_days.csv')

In [17]:
OnTime_by_days.head(20)

,Date_orders,OnTime
0,01/01/21,80.00
1,02/01/21,85.71
2,03/01/21,65.00
3,04/01/21,87.50
4,05/01/21,66.67
5,06/01/21,61.90
6,07/01/21,62.07
7,08/01/21,75.86
8,09/01/21,80.00
9,10/01/21,85.11


разрезы по перевозчикам, по типу доставки, по кластерам склада, по таймслотам:

по перевозчикам:

In [21]:
query = """

SELECT
    carrier_name, 
    ROUND(COUNT(CASE WHEN result_data_order == planned_data_order THEN number_order END)/COUNT(DISTINCT number_order)*100, 2) AS OnTime
FROM
    (
    SELECT
        toDateOrNull(concat(splitByString('-', assumeNotNull(date_order))[1], '-',
                            splitByString('-', assumeNotNull(date_order))[3], '-',
                            splitByString('-', assumeNotNull(date_order))[2])) as date_order,

        toDateOrNull(concat(splitByString('-', assumeNotNull(planned_data_order))[1], '-',
                            splitByString('-', assumeNotNull(planned_data_order))[3], '-',
                            splitByString('-', assumeNotNull(planned_data_order))[2])) as planned_data_order,

        toDateOrNull(concat(splitByString('-', assumeNotNull(result_data_order))[1], '-',
                            splitByString('-', assumeNotNull(result_data_order))[3], '-',
                            splitByString('-', assumeNotNull(result_data_order))[2])) as result_data_order,
        carrier_name, customer_cluster, delivery_type, number_order, seller_ID, warehouse_cluster, timeslot_number
    FROM default.logistics_online
    WHERE order_status = 'Доставлен' -- оставляем только фактически доставленные заказы
        AND date_order IS NOT NULL  -- избавляемся от данных без дат оформления заказов (так как такие данные невозможно анализирвать в динамике)
        AND date_order < result_data_order --учитываем возможный баг
        AND date_order < planned_data_order
        AND date_order != '1970-01-01'  -- учитываем конвертацию NaN в 1 января 1970
        AND planned_data_order != '1970-01-01'
        AND result_data_order != '1970-01-01'
    ) As tabl1
GROUP BY carrier_name
ORDER BY carrier_name ASC

"""
OnTime_by_carrier_name = ph.read_clickhouse(query, connection=connection)
OnTime_by_carrier_name

,carrier_name,OnTime
0,3PL 1,13.32
1,3PL 2,13.51
2,3PL 3,29.79
3,3PL 4,23.33
4,Самодоставка,23.94


у перевозчика 3PL 3 самый высокий процент доставки в срок, а у 3PL 1 и 3PL 2 - очень низкие, дальше стоит изучить перевозчиков более подробно, на предмет количества заказов, их динамики, с целью выяснения причин таких низких показателей метрики

по типу доставки:

In [20]:
query = """

SELECT
    delivery_type, 
    ROUND(COUNT(CASE WHEN result_data_order == planned_data_order THEN number_order END)/COUNT(DISTINCT number_order)*100, 2) AS OnTime
FROM
    (
    SELECT
        toDateOrNull(concat(splitByString('-', assumeNotNull(date_order))[1], '-',
                            splitByString('-', assumeNotNull(date_order))[3], '-',
                            splitByString('-', assumeNotNull(date_order))[2])) as date_order,

        toDateOrNull(concat(splitByString('-', assumeNotNull(planned_data_order))[1], '-',
                            splitByString('-', assumeNotNull(planned_data_order))[3], '-',
                            splitByString('-', assumeNotNull(planned_data_order))[2])) as planned_data_order,

        toDateOrNull(concat(splitByString('-', assumeNotNull(result_data_order))[1], '-',
                            splitByString('-', assumeNotNull(result_data_order))[3], '-',
                            splitByString('-', assumeNotNull(result_data_order))[2])) as result_data_order,
        carrier_name, customer_cluster, delivery_type, number_order, seller_ID, warehouse_cluster, timeslot_number
    FROM default.logistics_online
    WHERE order_status = 'Доставлен' -- оставляем только фактически доставленные заказы
        AND date_order IS NOT NULL  -- избавляемся от данных без дат оформления заказов (так как такие данные невозможно анализирвать в динамике)
        AND date_order < result_data_order --учитываем возможный баг
        AND date_order < planned_data_order
        AND date_order != '1970-01-01'  -- учитываем конвертацию NaN в 1 января 1970
        AND planned_data_order != '1970-01-01'
        AND result_data_order != '1970-01-01'
    ) As tabl1
GROUP BY delivery_type


"""
OnTime_by_delivery_type = ph.read_clickhouse(query, connection=connection)
OnTime_by_delivery_type

,delivery_type,OnTime
0,Курьер,22.26
1,Самовывоз,13.43


по кластерам склада:

In [22]:
query = """

SELECT
    warehouse_cluster, 
    ROUND(COUNT(CASE WHEN result_data_order == planned_data_order THEN number_order END)/COUNT(DISTINCT number_order)*100, 2) AS OnTime
FROM
    (
    SELECT
        toDateOrNull(concat(splitByString('-', assumeNotNull(date_order))[1], '-',
                            splitByString('-', assumeNotNull(date_order))[3], '-',
                            splitByString('-', assumeNotNull(date_order))[2])) as date_order,

        toDateOrNull(concat(splitByString('-', assumeNotNull(planned_data_order))[1], '-',
                            splitByString('-', assumeNotNull(planned_data_order))[3], '-',
                            splitByString('-', assumeNotNull(planned_data_order))[2])) as planned_data_order,

        toDateOrNull(concat(splitByString('-', assumeNotNull(result_data_order))[1], '-',
                            splitByString('-', assumeNotNull(result_data_order))[3], '-',
                            splitByString('-', assumeNotNull(result_data_order))[2])) as result_data_order,
        carrier_name, customer_cluster, delivery_type, number_order, seller_ID, warehouse_cluster, timeslot_number
    FROM default.logistics_online
    WHERE order_status = 'Доставлен' -- оставляем только фактически доставленные заказы
        AND date_order IS NOT NULL  -- избавляемся от данных без дат оформления заказов (так как такие данные невозможно анализирвать в динамике)
        AND date_order < result_data_order --учитываем возможный баг
        AND date_order < planned_data_order
        AND date_order != '1970-01-01'  -- учитываем конвертацию NaN в 1 января 1970
        AND planned_data_order != '1970-01-01'
        AND result_data_order != '1970-01-01'
    ) As tabl1
GROUP BY warehouse_cluster
ORDER BY warehouse_cluster ASC

"""
OnTime_by_warehouse_cluster = ph.read_clickhouse(query, connection=connection)
OnTime_by_warehouse_cluster

,warehouse_cluster,OnTime
0,Кластер Екатеринбург,5.28
1,Кластер Казань,12.38
2,Кластер Калининград,25.29
3,Кластер Краснодар,12.13
4,Кластер Москва,24.12
5,Кластер Московская область,26.08
6,Кластер Новосибирск,9.57
7,Кластер Санкт-Петербург,17.45
8,Кластер Северо-Запад,13.01
9,Кластер Хабаровск,57.14


больше всех доставляет в срок склад Хабаровска, склад Екатеринбурга за рассматриваемый период доставил вовремя меньше всего заказов

по таймслотам:

In [23]:
query = """

SELECT
    timeslot_number, 
    ROUND(COUNT(CASE WHEN result_data_order == planned_data_order THEN number_order END)/COUNT(DISTINCT number_order)*100, 2) AS OnTime
FROM
    (
    SELECT
        toDateOrNull(concat(splitByString('-', assumeNotNull(date_order))[1], '-',
                            splitByString('-', assumeNotNull(date_order))[3], '-',
                            splitByString('-', assumeNotNull(date_order))[2])) as date_order,

        toDateOrNull(concat(splitByString('-', assumeNotNull(planned_data_order))[1], '-',
                            splitByString('-', assumeNotNull(planned_data_order))[3], '-',
                            splitByString('-', assumeNotNull(planned_data_order))[2])) as planned_data_order,

        toDateOrNull(concat(splitByString('-', assumeNotNull(result_data_order))[1], '-',
                            splitByString('-', assumeNotNull(result_data_order))[3], '-',
                            splitByString('-', assumeNotNull(result_data_order))[2])) as result_data_order,
        carrier_name, customer_cluster, delivery_type, number_order, seller_ID, warehouse_cluster, timeslot_number
    FROM default.logistics_online
    WHERE order_status = 'Доставлен' -- оставляем только фактически доставленные заказы
        AND date_order IS NOT NULL  -- избавляемся от данных без дат оформления заказов (так как такие данные невозможно анализирвать в динамике)
        AND date_order < result_data_order --учитываем возможный баг
        AND date_order < planned_data_order
        AND date_order != '1970-01-01'  -- учитываем конвертацию NaN в 1 января 1970
        AND planned_data_order != '1970-01-01'
        AND result_data_order != '1970-01-01'
    ) As tabl1
GROUP BY timeslot_number


"""
OnTime_by_timeslot_number = ph.read_clickhouse(query, connection=connection)
OnTime_by_timeslot_number

,timeslot_number,OnTime
0,1,17.92


проверим были ли данные по другим таймслотам, посчитав количество строк, где таймслот был больше 1

In [24]:
query = """
SELECT
    COUNT() AS timeslot_number_more_1
FROM
(
    SELECT *
    FROM default.logistics_online
    WHERE order_status = 'Доставлен' 
    AND date_order IS NOT NULL 
    AND date_order < result_data_order
    AND date_order < planned_data_order

 ) AS tabl1
 
WHERE timeslot_number > 1 

"""
df = ph.read_clickhouse(query, connection=connection)
df

,timeslot_number_more_1
0,0


### Задача 2.

Посчитать PromisedClick2Delivery - метрика обещанного срока доставки (от даты заказа до первой плановой даты доставки), в сутках.

 - Необходима динамика по дням, неделям, месяцам;

 - Необходимо сравнение перевозчиков;

 - Необходимо понимание уровня метрики из кластера в кластер.

In [25]:
query = """
SELECT
    ROUND(AVG(PromisedClick2Delivery), 0) AS PromisedClick2Delivery
FROM 
    (
    SELECT
        date_order,
        dateDiff('day', date_order, planned_data_order) AS PromisedClick2Delivery
    FROM 
        (
            SELECT
        toDateOrNull(concat(splitByString('-', assumeNotNull(date_order))[1], '-',
                            splitByString('-', assumeNotNull(date_order))[3], '-',
                            splitByString('-', assumeNotNull(date_order))[2])) as date_order,

        toDateOrNull(concat(splitByString('-', assumeNotNull(planned_data_order))[1], '-',
                            splitByString('-', assumeNotNull(planned_data_order))[3], '-',
                            splitByString('-', assumeNotNull(planned_data_order))[2])) as planned_data_order,

        toDateOrNull(concat(splitByString('-', assumeNotNull(result_data_order))[1], '-',
                            splitByString('-', assumeNotNull(result_data_order))[3], '-',
                            splitByString('-', assumeNotNull(result_data_order))[2])) as result_data_order,
                carrier_name, customer_cluster, delivery_type, number_order, seller_ID, warehouse_cluster, timeslot_number
            FROM default.logistics_online
            WHERE order_status = 'Доставлен' 
                AND date_order IS NOT NULL 
                AND date_order < result_data_order
                AND date_order < planned_data_order
                AND date_order != '1970-01-01' 
                AND planned_data_order != '1970-01-01'
                AND result_data_order != '1970-01-01'
        ) AS tabl1
    WHERE timeslot_number =1 -- учитываем, что нам нужна только первая плановая дата, хотя уже известно, что в нашем датасете есть только этот вид таймслотов
    ) AS tabl2

"""
task2 = ph.read_clickhouse(query, connection=connection)
task2

,PromisedClick2Delivery
0,8.0


динамика по дням, неделям, месяцам:

в зависимости от нужного разреза, выбираем поле для группировки в операторах SELECT, GROUP BY и ORDER BY:

    SELECT
        toStartOfMonth(date_order) AS Month_order,
        toStartOfWeek(date_order) AS Week_order,
        date_order,
        ROUND(AVG(PromisedClick2Delivery), 0) AS PromisedClick2Delivery
    FROM 
        (
        SELECT
            date_order,
            dateDiff('day', date_order, planned_data_order) AS PromisedClick2Delivery
        FROM 
            (
                SELECT
                    toDateOrNull(concat(splitByString('-', assumeNotNull(date_order))[1], '-',
                                        splitByString('-', assumeNotNull(date_order))[3], '-',
                                        splitByString('-', assumeNotNull(date_order))[2])) as date_order,

                    toDateOrNull(concat(splitByString('-', assumeNotNull(planned_data_order))[1], '-',
                                        splitByString('-', assumeNotNull(planned_data_order))[3], '-',
                                        splitByString('-', assumeNotNull(planned_data_order))[2])) as planned_data_order,

                    toDateOrNull(concat(splitByString('-', assumeNotNull(result_data_order))[1], '-',
                                        splitByString('-', assumeNotNull(result_data_order))[3], '-',
                                        splitByString('-', assumeNotNull(result_data_order))[2])) as result_data_order,
                    carrier_name, customer_cluster, delivery_type, number_order, seller_ID, warehouse_cluster, timeslot_number
                FROM default.logistics_online
                WHERE order_status = 'Доставлен' 
                    AND date_order IS NOT NULL 
                    AND date_order < result_data_order
                    AND date_order < planned_data_order
                    AND date_order != '1970-01-01' 
                    AND planned_data_order != '1970-01-01'
                    AND result_data_order != '1970-01-01'
            ) AS tabl1
        WHERE timeslot_number =1
        ) AS tabl2 
    GROUP BY 
            Month_order,
            Week_order,
            date_order,
    ORDER BY Month_order ASC,
             Week_order ASC,
             date_order ASC

динамика по дням:

In [26]:
PromisedClick2Delivery_by_days = pd.read_csv('PromisedClick2Delivery_by_days.csv')

In [27]:
PromisedClick2Delivery_by_days

,date_order,PromisedClick2Delivery
0,01/01/21,9
1,02/01/21,9
2,03/01/21,9
3,04/01/21,9
4,05/01/21,9
...,...,...
68,10/03/21,6
69,11/03/21,6
70,12/03/21,6
71,13/03/21,5


динамика по неделям:

In [28]:
PromisedClick2Delivery_by_weeks = pd.read_csv('PromisedClick2Delivery_by_weeks.csv')

In [29]:
PromisedClick2Delivery_by_weeks

,Week_order,PromisedClick2Delivery
0,27/12/20,9
1,03/01/21,8
2,10/01/21,8
3,17/01/21,8
4,24/01/21,8
5,31/01/21,8
6,07/02/21,8
7,14/02/21,8
8,21/02/21,8
9,28/02/21,8


динамика по месяцам:

In [30]:
PromisedClick2Delivery_by_months = pd.read_csv('PromisedClick2Delivery_by_months.csv')

In [31]:
PromisedClick2Delivery_by_months

,Month_order,PromisedClick2Delivery
0,01/01/21,8
1,01/02/21,8
2,01/03/21,7


данные во временных разрезах достаточно однородны, без выбросов

сравнение перевозчиков:

In [32]:
query = """
SELECT
    carrier_name, 
    ROUND(AVG(PromisedClick2Delivery), 0) AS PromisedClick2Delivery
FROM 
    (
    
    SELECT
        date_order,
        dateDiff('day', date_order, planned_data_order) AS PromisedClick2Delivery,
        carrier_name, warehouse_cluster, customer_cluster
        
    FROM 
        (
            SELECT
                toDateOrNull(concat(splitByString('-', assumeNotNull(date_order))[1], '-',
                                    splitByString('-', assumeNotNull(date_order))[3], '-',
                                    splitByString('-', assumeNotNull(date_order))[2])) as date_order,

                toDateOrNull(concat(splitByString('-', assumeNotNull(planned_data_order))[1], '-',
                                    splitByString('-', assumeNotNull(planned_data_order))[3], '-',
                                    splitByString('-', assumeNotNull(planned_data_order))[2])) as planned_data_order,

                toDateOrNull(concat(splitByString('-', assumeNotNull(result_data_order))[1], '-',
                                    splitByString('-', assumeNotNull(result_data_order))[3], '-',
                                    splitByString('-', assumeNotNull(result_data_order))[2])) as result_data_order,
                carrier_name, customer_cluster, delivery_type, number_order, seller_ID, warehouse_cluster, timeslot_number
            FROM default.logistics_online
            WHERE order_status = 'Доставлен' 
                AND date_order IS NOT NULL 
                AND date_order < result_data_order
                AND date_order < planned_data_order
                AND date_order != '1970-01-01' 
                AND planned_data_order != '1970-01-01'
                AND result_data_order != '1970-01-01'
        ) AS tabl1
    WHERE timeslot_number =1
    ) AS tabl2
GROUP BY carrier_name
ORDER BY carrier_name ASC


"""
PromisedClick2Delivery_by_carrier_name = ph.read_clickhouse(query, connection=connection)
PromisedClick2Delivery_by_carrier_name

,carrier_name,PromisedClick2Delivery
0,3PL 1,8.0
1,3PL 2,8.0
2,3PL 3,8.0
3,3PL 4,8.0
4,Самодоставка,8.0


 метрика обещанного срока доставки однородна в разрезе перевозчиков

метрика из кластера в кластер:

In [33]:
query = """
SELECT
    warehouse_cluster, 
    ROUND(AVG(PromisedClick2Delivery), 0) AS PromisedClick2Delivery
FROM 
    (
    
    SELECT
        date_order,
        dateDiff('day', date_order, planned_data_order) AS PromisedClick2Delivery,
        carrier_name, warehouse_cluster, customer_cluster
        
    FROM 
        (
            SELECT
                toDateOrNull(concat(splitByString('-', assumeNotNull(date_order))[1], '-',
                                    splitByString('-', assumeNotNull(date_order))[3], '-',
                                    splitByString('-', assumeNotNull(date_order))[2])) as date_order,

                toDateOrNull(concat(splitByString('-', assumeNotNull(planned_data_order))[1], '-',
                                    splitByString('-', assumeNotNull(planned_data_order))[3], '-',
                                    splitByString('-', assumeNotNull(planned_data_order))[2])) as planned_data_order,

                toDateOrNull(concat(splitByString('-', assumeNotNull(result_data_order))[1], '-',
                                    splitByString('-', assumeNotNull(result_data_order))[3], '-',
                                    splitByString('-', assumeNotNull(result_data_order))[2])) as result_data_order,
                carrier_name, customer_cluster, delivery_type, number_order, seller_ID, warehouse_cluster, timeslot_number
            FROM default.logistics_online
            WHERE order_status = 'Доставлен' 
                AND date_order IS NOT NULL 
                AND date_order < result_data_order
                AND date_order < planned_data_order
                AND date_order != '1970-01-01' 
                AND planned_data_order != '1970-01-01'
                AND result_data_order != '1970-01-01'
        ) AS tabl1
    WHERE timeslot_number =1
    ) AS tabl2
GROUP BY warehouse_cluster
ORDER BY warehouse_cluster

"""
PromisedClick2Delivery_by_warehouse_cluster = ph.read_clickhouse(query, connection=connection)
PromisedClick2Delivery_by_warehouse_cluster

,warehouse_cluster,PromisedClick2Delivery
0,Кластер Екатеринбург,8.0
1,Кластер Казань,8.0
2,Кластер Калининград,8.0
3,Кластер Краснодар,8.0
4,Кластер Москва,8.0
5,Кластер Московская область,8.0
6,Кластер Новосибирск,8.0
7,Кластер Санкт-Петербург,8.0
8,Кластер Северо-Запад,8.0
9,Кластер Хабаровск,7.0


обещанный срок по складским кластерам одинаков за исключением склада в Хабаровске

In [34]:
query = """
SELECT
    customer_cluster,
    ROUND(AVG(PromisedClick2Delivery), 0) AS PromisedClick2Delivery
FROM 
    (
    
    SELECT
        date_order,
        dateDiff('day', date_order, planned_data_order) AS PromisedClick2Delivery,
        carrier_name, warehouse_cluster, customer_cluster
        
    FROM 
        (
            SELECT
                toDateOrNull(concat(splitByString('-', assumeNotNull(date_order))[1], '-',
                                    splitByString('-', assumeNotNull(date_order))[3], '-',
                                    splitByString('-', assumeNotNull(date_order))[2])) as date_order,

                toDateOrNull(concat(splitByString('-', assumeNotNull(planned_data_order))[1], '-',
                                    splitByString('-', assumeNotNull(planned_data_order))[3], '-',
                                    splitByString('-', assumeNotNull(planned_data_order))[2])) as planned_data_order,

                toDateOrNull(concat(splitByString('-', assumeNotNull(result_data_order))[1], '-',
                                    splitByString('-', assumeNotNull(result_data_order))[3], '-',
                                    splitByString('-', assumeNotNull(result_data_order))[2])) as result_data_order,
                carrier_name, customer_cluster, delivery_type, number_order, seller_ID, warehouse_cluster, timeslot_number
            FROM default.logistics_online
            WHERE order_status = 'Доставлен' 
                AND date_order IS NOT NULL 
                AND date_order < result_data_order
                AND date_order < planned_data_order
                AND date_order != '1970-01-01' 
                AND planned_data_order != '1970-01-01'
                AND result_data_order != '1970-01-01'
        ) AS tabl1
    WHERE timeslot_number =1
    ) AS tabl2
GROUP BY customer_cluster
ORDER BY customer_cluster ASC

"""
PromisedClick2Delivery_by_customer_cluster = ph.read_clickhouse(query, connection=connection)
PromisedClick2Delivery_by_customer_cluster

,customer_cluster,PromisedClick2Delivery
0,Кластер Екатеринбург,8.0
1,Кластер Казань,8.0
2,Кластер Калининград,8.0
3,Кластер Краснодар,8.0
4,Кластер Москва,8.0
5,Кластер Московская область,8.0
6,Кластер Новосибирск,8.0
7,Кластер Санкт-Петербург,8.0
8,Кластер Северо-Запад,8.0
9,Кластер Хабаровск,8.0


покупателям из Якутии обещают самые короткие сроки

### Задача 3.

Посчитать Click2Delivery - метрика фактического срока доставки (от даты заказа до момента вручения), в сутках.

 - Необходимо сравнение с метрикой PromisedClick2Delivery;

 - Необходимо понимание кластеров, где заказы доставляются быстрее, чем мы обещаем клиентам;

In [35]:
query = """
SELECT
    ROUND(AVG(Click2Delivery), 0) AS Click2Delivery,
    ROUND(AVG(PromisedClick2Delivery), 0) AS PromisedClick2Delivery  
FROM 
    (
    SELECT
        date_order,
        dateDiff('day', date_order, planned_data_order) AS PromisedClick2Delivery,
        dateDiff('day', date_order, result_data_order) AS Click2Delivery
FROM 
    (
        SELECT
                toDateOrNull(concat(splitByString('-', assumeNotNull(date_order))[1], '-',
                                    splitByString('-', assumeNotNull(date_order))[3], '-',
                                    splitByString('-', assumeNotNull(date_order))[2])) as date_order,

                toDateOrNull(concat(splitByString('-', assumeNotNull(planned_data_order))[1], '-',
                                    splitByString('-', assumeNotNull(planned_data_order))[3], '-',
                                    splitByString('-', assumeNotNull(planned_data_order))[2])) as planned_data_order,

                toDateOrNull(concat(splitByString('-', assumeNotNull(result_data_order))[1], '-',
                                    splitByString('-', assumeNotNull(result_data_order))[3], '-',
                                    splitByString('-', assumeNotNull(result_data_order))[2])) as result_data_order,
                carrier_name, customer_cluster, delivery_type, number_order, seller_ID, warehouse_cluster, timeslot_number
        FROM default.logistics_online
        WHERE order_status = 'Доставлен' 
            AND date_order IS NOT NULL 
            AND date_order < result_data_order
            AND date_order < planned_data_order
            AND date_order != '1970-01-01' 
            AND planned_data_order != '1970-01-01'
            AND result_data_order != '1970-01-01'
    ) AS tabl1
) AS tabl2

"""
task3 = ph.read_clickhouse(query, connection=connection)
task3

,Click2Delivery,PromisedClick2Delivery
0,6.0,8.0


фактический срок доставки на 2 дня короче обещанного

кластеры, где заказы доставляются быстрее, чем обещали клиентам:

In [36]:
query = """
SELECT
    warehouse_cluster, 
    ROUND(AVG(Click2Delivery), 0) AS Click2Delivery,  
    ROUND(AVG(PromisedClick2Delivery), 0) AS PromisedClick2Delivery

FROM 
    (
    SELECT
        customer_cluster, warehouse_cluster, 
        dateDiff('day', date_order, result_data_order) AS Click2Delivery,
        dateDiff('day', date_order, planned_data_order) AS PromisedClick2Delivery
FROM 
    (
        SELECT
                toDateOrNull(concat(splitByString('-', assumeNotNull(date_order))[1], '-',
                                    splitByString('-', assumeNotNull(date_order))[3], '-',
                                    splitByString('-', assumeNotNull(date_order))[2])) as date_order,

                toDateOrNull(concat(splitByString('-', assumeNotNull(planned_data_order))[1], '-',
                                    splitByString('-', assumeNotNull(planned_data_order))[3], '-',
                                    splitByString('-', assumeNotNull(planned_data_order))[2])) as planned_data_order,

                toDateOrNull(concat(splitByString('-', assumeNotNull(result_data_order))[1], '-',
                                    splitByString('-', assumeNotNull(result_data_order))[3], '-',
                                    splitByString('-', assumeNotNull(result_data_order))[2])) as result_data_order,
                carrier_name, customer_cluster, delivery_type, number_order, seller_ID, warehouse_cluster, timeslot_number
        FROM default.logistics_online
        WHERE order_status = 'Доставлен' 
            AND date_order IS NOT NULL 
            AND date_order < result_data_order
            AND date_order < planned_data_order
            AND date_order != '1970-01-01' 
            AND planned_data_order != '1970-01-01'
            AND result_data_order != '1970-01-01'
    ) AS tabl1
) AS tabl2
GROUP BY warehouse_cluster
ORDER BY warehouse_cluster ASC


"""
Click2Delivery_warehouse_cluster = ph.read_clickhouse(query, connection=connection)
Click2Delivery_warehouse_cluster

,warehouse_cluster,Click2Delivery,PromisedClick2Delivery
0,Кластер Екатеринбург,4.0,8.0
1,Кластер Казань,6.0,8.0
2,Кластер Калининград,7.0,8.0
3,Кластер Краснодар,6.0,8.0
4,Кластер Москва,7.0,8.0
5,Кластер Московская область,7.0,8.0
6,Кластер Новосибирск,6.0,8.0
7,Кластер Санкт-Петербург,6.0,8.0
8,Кластер Северо-Запад,6.0,8.0
9,Кластер Хабаровск,8.0,7.0


In [37]:
Click2Delivery_warehouse_cluster['faster than promised'] = Click2Delivery_warehouse_cluster.PromisedClick2Delivery-Click2Delivery_warehouse_cluster.Click2Delivery

In [38]:
Click2Delivery_warehouse_cluster

,warehouse_cluster,Click2Delivery,PromisedClick2Delivery,faster than promised
0,Кластер Екатеринбург,4.0,8.0,4.0
1,Кластер Казань,6.0,8.0,2.0
2,Кластер Калининград,7.0,8.0,1.0
3,Кластер Краснодар,6.0,8.0,2.0
4,Кластер Москва,7.0,8.0,1.0
5,Кластер Московская область,7.0,8.0,1.0
6,Кластер Новосибирск,6.0,8.0,2.0
7,Кластер Санкт-Петербург,6.0,8.0,2.0
8,Кластер Северо-Запад,6.0,8.0,2.0
9,Кластер Хабаровск,8.0,7.0,-1.0


кластеры склада, где заказы доставляются быстрее, чем обещали клиентам - кластер Екатеринбурга (на 4 дня быстрей), кластер Центр (на 3 дня быстрей), замыкает тройку лидеров кластеры Казани, Краснодара, Новосибирска, Санкт-Петербурга (на 2 дня быстрей). Склады Москвы и Подмосковья лучше всего прогнозируют сроки доставки. Кластер Хабаровска излишне оптимистично оценивает свои возможности по доставке (в среднем доставляют на 1 день позже обещанного)

In [39]:
query = """
SELECT
    customer_cluster,
    ROUND(AVG(Click2Delivery), 0) AS Click2Delivery,
    ROUND(AVG(PromisedClick2Delivery), 0) AS PromisedClick2Delivery
FROM 
    (
    SELECT
        customer_cluster, warehouse_cluster, 
        dateDiff('day', date_order, result_data_order) AS Click2Delivery,
        dateDiff('day', date_order, planned_data_order) AS PromisedClick2Delivery
FROM 
    (
        SELECT
                toDateOrNull(concat(splitByString('-', assumeNotNull(date_order))[1], '-',
                                    splitByString('-', assumeNotNull(date_order))[3], '-',
                                    splitByString('-', assumeNotNull(date_order))[2])) as date_order,

                toDateOrNull(concat(splitByString('-', assumeNotNull(planned_data_order))[1], '-',
                                    splitByString('-', assumeNotNull(planned_data_order))[3], '-',
                                    splitByString('-', assumeNotNull(planned_data_order))[2])) as planned_data_order,

                toDateOrNull(concat(splitByString('-', assumeNotNull(result_data_order))[1], '-',
                                    splitByString('-', assumeNotNull(result_data_order))[3], '-',
                                    splitByString('-', assumeNotNull(result_data_order))[2])) as result_data_order,
                carrier_name, customer_cluster, delivery_type, number_order, seller_ID, warehouse_cluster, timeslot_number
        FROM default.logistics_online
        WHERE order_status = 'Доставлен' 
            AND date_order IS NOT NULL 
            AND date_order < result_data_order
            AND date_order < planned_data_order
            AND date_order != '1970-01-01' 
            AND planned_data_order != '1970-01-01'
            AND result_data_order != '1970-01-01'
    ) AS tabl1
) AS tabl2
GROUP BY customer_cluster
ORDER BY customer_cluster ASC


"""
Click2Delivery_customer_cluster = ph.read_clickhouse(query, connection=connection)
Click2Delivery_customer_cluster

,customer_cluster,Click2Delivery,PromisedClick2Delivery
0,Кластер Екатеринбург,5.0,8.0
1,Кластер Казань,6.0,8.0
2,Кластер Калининград,8.0,8.0
3,Кластер Краснодар,6.0,8.0
4,Кластер Москва,7.0,8.0
5,Кластер Московская область,6.0,8.0
6,Кластер Новосибирск,6.0,8.0
7,Кластер Санкт-Петербург,6.0,8.0
8,Кластер Северо-Запад,6.0,8.0
9,Кластер Хабаровск,9.0,8.0


In [40]:
Click2Delivery_customer_cluster['faster than promised'] = Click2Delivery_customer_cluster.PromisedClick2Delivery-Click2Delivery_customer_cluster.Click2Delivery

In [41]:
Click2Delivery_customer_cluster

,customer_cluster,Click2Delivery,PromisedClick2Delivery,faster than promised
0,Кластер Екатеринбург,5.0,8.0,3.0
1,Кластер Казань,6.0,8.0,2.0
2,Кластер Калининград,8.0,8.0,0.0
3,Кластер Краснодар,6.0,8.0,2.0
4,Кластер Москва,7.0,8.0,1.0
5,Кластер Московская область,6.0,8.0,2.0
6,Кластер Новосибирск,6.0,8.0,2.0
7,Кластер Санкт-Петербург,6.0,8.0,2.0
8,Кластер Северо-Запад,6.0,8.0,2.0
9,Кластер Хабаровск,9.0,8.0,-1.0


кластеры принадлежности клиента, где заказы доставляются быстрее, чем обещали - кластер Екатеринбурга (на 3 дня быстрей), кластеры Казани, Краснодара, Подмосковья, Новосибирска, Санкт-Петербурга, Северо-Запада и Центра (на 2 дня быстрей). В обещанный срок доставляет заказы кластер Калининграда. С задержской на 1 день доставляет кластер Хабаровска и на 3 дня - кластер Якутии (в Якутии нет складского кластера)

### Задача 4.

Посчитать GMV Accepted - метрика совокупного объема продаж (считается по заказам, оформленным в указанный день), в рублях.

 - Необходимо видеть изменение метрики по дням;

 - Необходимы разрезы по перевозчикам, по типу доставки, по кластерам склада;

In [42]:
query = """
SELECT
    SUM(sum_order) AS GMV_Accepted 
FROM 
    (
    SELECT
        date_order,
        number_order,
        sum_order
    FROM 
        (
            SELECT
                toDateOrNull(concat(splitByString('-', assumeNotNull(date_order))[1], '-',
                                    splitByString('-', assumeNotNull(date_order))[3], '-',
                                    splitByString('-', assumeNotNull(date_order))[2])) as date_order,

                toDateOrNull(concat(splitByString('-', assumeNotNull(planned_data_order))[1], '-',
                                    splitByString('-', assumeNotNull(planned_data_order))[3], '-',
                                    splitByString('-', assumeNotNull(planned_data_order))[2])) as planned_data_order,

                toDateOrNull(concat(splitByString('-', assumeNotNull(result_data_order))[1], '-',
                                    splitByString('-', assumeNotNull(result_data_order))[3], '-',
                                    splitByString('-', assumeNotNull(result_data_order))[2])) as result_data_order,
                carrier_name, customer_cluster, delivery_type, number_order, seller_ID, warehouse_cluster, timeslot_number
            FROM default.logistics_online
            WHERE order_status = 'Доставлен' 
                AND date_order IS NOT NULL 
                AND date_order < result_data_order
                AND date_order < planned_data_order
                AND date_order != '1970-01-01' 
                AND planned_data_order != '1970-01-01'
                AND result_data_order != '1970-01-01'

    ) AS L
    JOIN -- нам нужны только данные на пересечении двух таблиц, поэтому нам нужен inner join
    (
            SELECT
                number_order,
                SUM(sum_product) AS sum_order
            FROM 
                (
                SELECT 
                    number_order, product_ID,
                    product_price*amount AS sum_product
                FROM default.product_online
                ) AS tabl1
            GROUP BY number_order
            ) AS R
    ON L.number_order=R.number_order
    ) AS tabl2

"""
task4 = ph.read_clickhouse(query, connection=connection)
task4

,GMV_Accepted
0,80528694.41


изменение метрики по дням:

    SELECT
        date_order,
        SUM(sum_order) AS GMV_Accepted 

    FROM 
        (
        SELECT
            date_order,
            number_order,
            sum_order
        FROM 
            (
                SELECT
                toDateOrNull(concat(splitByString('-', assumeNotNull(date_order))[1], '-',
                                    splitByString('-', assumeNotNull(date_order))[3], '-',
                                    splitByString('-', assumeNotNull(date_order))[2])) as date_order,

                toDateOrNull(concat(splitByString('-', assumeNotNull(planned_data_order))[1], '-',
                                    splitByString('-', assumeNotNull(planned_data_order))[3], '-',
                                    splitByString('-', assumeNotNull(planned_data_order))[2])) as planned_data_order,

                toDateOrNull(concat(splitByString('-', assumeNotNull(result_data_order))[1], '-',
                                    splitByString('-', assumeNotNull(result_data_order))[3], '-',
                                    splitByString('-', assumeNotNull(result_data_order))[2])) as result_data_order,
                carrier_name, customer_cluster, delivery_type, number_order, seller_ID, warehouse_cluster, timeslot_number
                FROM default.logistics_online
                WHERE order_status = 'Доставлен' 
                    AND date_order IS NOT NULL 
                    AND date_order < result_data_order
                    AND date_order < planned_data_order
                    AND date_order != '1970-01-01' 
                    AND planned_data_order != '1970-01-01'
                    AND result_data_order != '1970-01-01'
        ) AS L
        JOIN -- нам нужны только данные на пересечении двух таблиц, поэтому нам нужен inner join
        (
                SELECT
                    number_order,
                    SUM(sum_product) AS sum_order
                FROM 
                    (
                    SELECT 
                        number_order, product_ID,
                        product_price*amount AS sum_product
                    FROM default.product_online
                    ) AS tabl1
                GROUP BY number_order
                ) AS R
        ON L.number_order=R.number_order
        ) AS tabl2
    GROUP BY date_order
    ORDER BY date_order ASC

In [43]:
GMV_Accepted_by_days = pd.read_csv('GMV_Accepted_by_days.csv')

In [44]:
GMV_Accepted_by_days

,date_order,GMV_Accepted
0,01/01/21,165253.86
1,02/01/21,219164.61
2,03/01/21,377629.99
3,04/01/21,352917.24
4,05/01/21,556032.63
...,...,...
68,10/03/21,817145.20
69,11/03/21,614406.20
70,12/03/21,264715.44
71,13/03/21,166059.53


разрезы по перевозчикам, по типу доставки, по кластерам склада:

разрезы по перевозчикам:

In [48]:
query = """
SELECT
    carrier_name, 
    SUM(sum_order) AS GMV_Accepted 
FROM 
    (
    SELECT
        date_order,
        number_order,
        sum_order,
        carrier_name, customer_cluster, delivery_type, warehouse_cluster
    FROM 
        (
            SELECT
                toDateOrNull(concat(splitByString('-', assumeNotNull(date_order))[1], '-',
                                    splitByString('-', assumeNotNull(date_order))[3], '-',
                                    splitByString('-', assumeNotNull(date_order))[2])) as date_order,

                toDateOrNull(concat(splitByString('-', assumeNotNull(planned_data_order))[1], '-',
                                    splitByString('-', assumeNotNull(planned_data_order))[3], '-',
                                    splitByString('-', assumeNotNull(planned_data_order))[2])) as planned_data_order,

                toDateOrNull(concat(splitByString('-', assumeNotNull(result_data_order))[1], '-',
                                    splitByString('-', assumeNotNull(result_data_order))[3], '-',
                                    splitByString('-', assumeNotNull(result_data_order))[2])) as result_data_order,
                carrier_name, customer_cluster, delivery_type, number_order, seller_ID, warehouse_cluster, timeslot_number
            FROM default.logistics_online
            WHERE order_status = 'Доставлен' 
                AND date_order IS NOT NULL 
                AND date_order < result_data_order
                AND date_order < planned_data_order
                AND date_order != '1970-01-01' 
                AND planned_data_order != '1970-01-01'
                AND result_data_order != '1970-01-01'

    ) AS L
    JOIN -- нам нужны только данные на пересечении двух таблиц, поэтому нам нужен inner join
    (
            SELECT
                number_order,
                SUM(sum_product) AS sum_order
            FROM 
                (
                SELECT 
                    number_order, product_ID,
                    product_price*amount AS sum_product
                FROM default.product_online
                ) AS tabl1
            GROUP BY number_order
            ) AS R
    ON L.number_order=R.number_order
    ) AS tabl2
GROUP BY carrier_name
ORDER BY SUM(sum_order) DESC

"""
GMV_Accepted_carrier_name = ph.read_clickhouse(query, connection=connection)
GMV_Accepted_carrier_name

,carrier_name,GMV_Accepted
0,3PL 1,26313176.15
1,3PL 2,22967699.07
2,Самодоставка,22568657.87
3,3PL 3,5736341.29
4,3PL 4,2942820.03


на 3PL 1 приходится больше всего совокупного объема продаж, меньше всего приходится на 3PL 4

разрезы по типу доставки:

In [52]:
query = """
SELECT
    delivery_type, 
    SUM(sum_order) AS GMV_Accepted 
FROM 
    (
    SELECT
        date_order,
        number_order,
        sum_order,
        carrier_name, customer_cluster, delivery_type, warehouse_cluster
    FROM 
        (
            SELECT
                toDateOrNull(concat(splitByString('-', assumeNotNull(date_order))[1], '-',
                                    splitByString('-', assumeNotNull(date_order))[3], '-',
                                    splitByString('-', assumeNotNull(date_order))[2])) as date_order,

                toDateOrNull(concat(splitByString('-', assumeNotNull(planned_data_order))[1], '-',
                                    splitByString('-', assumeNotNull(planned_data_order))[3], '-',
                                    splitByString('-', assumeNotNull(planned_data_order))[2])) as planned_data_order,

                toDateOrNull(concat(splitByString('-', assumeNotNull(result_data_order))[1], '-',
                                    splitByString('-', assumeNotNull(result_data_order))[3], '-',
                                    splitByString('-', assumeNotNull(result_data_order))[2])) as result_data_order,
                carrier_name, customer_cluster, delivery_type, number_order, seller_ID, warehouse_cluster, timeslot_number
            FROM default.logistics_online
            WHERE order_status = 'Доставлен' 
                AND date_order IS NOT NULL 
                AND date_order < result_data_order
                AND date_order < planned_data_order
                AND date_order != '1970-01-01' 
                AND planned_data_order != '1970-01-01'
                AND result_data_order != '1970-01-01'

    ) AS L
    JOIN -- нам нужны только данные на пересечении двух таблиц, поэтому нам нужен inner join
    (
            SELECT
                number_order,
                SUM(sum_product) AS sum_order
            FROM 
                (
                SELECT 
                    number_order, product_ID,
                    product_price*amount AS sum_product
                FROM default.product_online
                ) AS tabl1
            GROUP BY number_order
            ) AS R
    ON L.number_order=R.number_order
    ) AS tabl2
GROUP BY delivery_type
ORDER BY SUM(sum_order) DESC

"""
GMV_Accepted_delivery_type = ph.read_clickhouse(query, connection=connection)
GMV_Accepted_delivery_type

,delivery_type,GMV_Accepted
0,Курьер,41754703.90
1,Самовывоз,38773990.51


разрезы по кластерам склада:

In [53]:
query = """
SELECT
    warehouse_cluster,
    SUM(sum_order) AS GMV_Accepted 
FROM 
    (
    SELECT
        date_order,
        number_order,
        sum_order,
        carrier_name, customer_cluster, delivery_type, warehouse_cluster
    FROM 
        (
            SELECT
                toDateOrNull(concat(splitByString('-', assumeNotNull(date_order))[1], '-',
                                    splitByString('-', assumeNotNull(date_order))[3], '-',
                                    splitByString('-', assumeNotNull(date_order))[2])) as date_order,

                toDateOrNull(concat(splitByString('-', assumeNotNull(planned_data_order))[1], '-',
                                    splitByString('-', assumeNotNull(planned_data_order))[3], '-',
                                    splitByString('-', assumeNotNull(planned_data_order))[2])) as planned_data_order,

                toDateOrNull(concat(splitByString('-', assumeNotNull(result_data_order))[1], '-',
                                    splitByString('-', assumeNotNull(result_data_order))[3], '-',
                                    splitByString('-', assumeNotNull(result_data_order))[2])) as result_data_order,
                carrier_name, customer_cluster, delivery_type, number_order, seller_ID, warehouse_cluster, timeslot_number
            FROM default.logistics_online
            WHERE order_status = 'Доставлен' 
                AND date_order IS NOT NULL 
                AND date_order < result_data_order
                AND date_order < planned_data_order
                AND date_order != '1970-01-01' 
                AND planned_data_order != '1970-01-01'
                AND result_data_order != '1970-01-01'

    ) AS L
    JOIN -- нам нужны только данные на пересечении двух таблиц, поэтому нам нужен inner join
    (
            SELECT
                number_order,
                SUM(sum_product) AS sum_order
            FROM 
                (
                SELECT 
                    number_order, product_ID,
                    product_price*amount AS sum_product
                FROM default.product_online
                ) AS tabl1
            GROUP BY number_order
            ) AS R
    ON L.number_order=R.number_order
    ) AS tabl2
GROUP BY warehouse_cluster
ORDER BY SUM(sum_order) DESC

"""
GMV_Accepted_warehouse_cluster = ph.read_clickhouse(query, connection=connection)
GMV_Accepted_warehouse_cluster

,warehouse_cluster,GMV_Accepted
0,Кластер Москва,25870647.13
1,Кластер Московская область,13087517.43
2,Кластер Казань,7868504.31
3,Кластер Екатеринбург,7419389.51
4,Кластер Краснодар,7073705.68
5,Кластер Санкт-Петербург,6976494.19
6,Кластер Новосибирск,4695119.80
7,Кластер Центр,3404147.91
8,Кластер Северо-Запад,2070416.11
9,Кластер Калининград,1946410.83


предсказуемо больше всего совокупного объема продаж приходится на складской кластер Москвы, далее идут кластеры Подмосковья и Казани, меньше всего дохода приносит кластер Хабаровска

### Задача 5.

Посчитать Return Rate - доля возвратов (отношение количества возвратов к заказам в этот день), в %.

 - Необходимо видеть изменение метрики по дням;

 - Необходимы разрезы по перевозчикам, по кластерам склада и клиента;

In [54]:
query = """
SELECT
    ROUND(AVG(Return_Rate), 2) AS Avg_Return_Rate
FROM
(
SELECT
    date_order,
    COUNT(L.number_order) AS orders, 
    COUNT(CASE WHEN R.number_order !=0 THEN R.number_order END) AS return_orders,
    COUNT(CASE WHEN R.number_order !=0 THEN R.number_order END)/COUNT(L.number_order)*100 AS Return_Rate 
FROM 
    (
    SELECT
                toDateOrNull(concat(splitByString('-', assumeNotNull(date_order))[1], '-',
                                    splitByString('-', assumeNotNull(date_order))[3], '-',
                                    splitByString('-', assumeNotNull(date_order))[2])) as date_order,

                toDateOrNull(concat(splitByString('-', assumeNotNull(planned_data_order))[1], '-',
                                    splitByString('-', assumeNotNull(planned_data_order))[3], '-',
                                    splitByString('-', assumeNotNull(planned_data_order))[2])) as planned_data_order,

                toDateOrNull(concat(splitByString('-', assumeNotNull(result_data_order))[1], '-',
                                    splitByString('-', assumeNotNull(result_data_order))[3], '-',
                                    splitByString('-', assumeNotNull(result_data_order))[2])) as result_data_order,
                carrier_name, customer_cluster, delivery_type, number_order, seller_ID, warehouse_cluster, timeslot_number
    FROM default.logistics_online
    WHERE order_status = 'Доставлен' 
        AND date_order IS NOT NULL 
        AND date_order < result_data_order
        AND date_order != '1970-01-01'  
        AND planned_data_order != '1970-01-01'
        AND result_data_order != '1970-01-01'
    ) AS L
LEFT JOIN -- отношение количества возвратов к заказам в день можно найти если в этот день заказы были, т е для объединения нам нужно использовать left join
    (
    SELECT DISTINCT number_order
    FROM default.returns_online
    ) AS R
ON L.number_order=R.number_order
GROUP BY date_order
ORDER BY date_order ASC
) AS tabl1   
"""
task5 = ph.read_clickhouse(query, connection=connection)
task5

,Avg_Return_Rate
0,5.29


изменение метрики по дням:


    SELECT
        date_order,
        COUNT(L.number_order) AS orders, 
        COUNT(CASE WHEN R.number_order !=0 THEN R.number_order END) AS return_orders,
        ROUND(COUNT(CASE WHEN R.number_order !=0 THEN R.number_order END)/COUNT(L.number_order)*100, 2) AS Return_Rate 
    FROM 
        (
        SELECT
                toDateOrNull(concat(splitByString('-', assumeNotNull(date_order))[1], '-',
                                    splitByString('-', assumeNotNull(date_order))[3], '-',
                                    splitByString('-', assumeNotNull(date_order))[2])) as date_order,

                toDateOrNull(concat(splitByString('-', assumeNotNull(planned_data_order))[1], '-',
                                    splitByString('-', assumeNotNull(planned_data_order))[3], '-',
                                    splitByString('-', assumeNotNull(planned_data_order))[2])) as planned_data_order,

                toDateOrNull(concat(splitByString('-', assumeNotNull(result_data_order))[1], '-',
                                    splitByString('-', assumeNotNull(result_data_order))[3], '-',
                                    splitByString('-', assumeNotNull(result_data_order))[2])) as result_data_order,
                carrier_name, customer_cluster, delivery_type, number_order, seller_ID, warehouse_cluster, timeslot_number
        FROM default.logistics_online
        WHERE order_status = 'Доставлен' 
            AND date_order IS NOT NULL 
            AND date_order < result_data_order
            AND date_order != '1970-01-01'  
            AND planned_data_order != '1970-01-01'
            AND result_data_order != '1970-01-01'
        ) AS L
    LEFT JOIN -- отношение количества возвратов к заказам в день можно найти если в этот день заказы были, т е для объединения нам нужно использовать left join
        (
        SELECT DISTINCT number_order
        FROM default.returns_online
        ) AS R
    ON L.number_order=R.number_order
    GROUP BY date_order
    ORDER BY date_order ASC


In [57]:
Return_Rate_by_days = pd.read_csv('Return_Rate_by_days.csv')

In [70]:
Return_Rate_by_days

,Date_orders,orders,return_orders,Return_Rate
0,01/01/21,10,0,0.0
1,02/01/21,14,1,7.1
2,03/01/21,20,2,10.0
3,04/01/21,24,1,4.2
4,05/01/21,21,1,4.8
...,...,...,...,...
68,10/03/21,40,0,0.0
69,11/03/21,26,1,3.8
70,12/03/21,10,0,0.0
71,13/03/21,10,0,0.0


In [69]:
Return_Rate_by_days.sort_values('Return_Rate', ascending=False)

,Date_orders,orders,return_orders,Return_Rate
26,27/01/21,50,7,14.0
20,21/01/21,22,3,13.6
39,09/02/21,76,8,10.5
7,08/01/21,29,3,10.3
2,03/01/21,20,2,10.0
...,...,...,...,...
19,20/01/21,33,0,0.0
16,17/01/21,31,0,0.0
13,14/01/21,24,0,0.0
11,12/01/21,31,0,0.0


есть дни с аномально высоким процентом возвратов, например,  27/01/21(14%), 21/01/21(13,6%), 09/02/21(10,5%), 08/01/21(10,3%) и 03/01/21(10%), необходимо разбираться дальше, что явилось причиной возвратов

разрезы по перевозчикам, по кластерам склада и клиента:

разрезы по перевозчикам:

In [71]:
query = """

SELECT
    carrier_name,
    ROUND(COUNT(CASE WHEN R.number_order !=0 THEN R.number_order END)/COUNT(L.number_order)*100, 2) AS Return_Rate 
FROM 
    (
    SELECT
                toDateOrNull(concat(splitByString('-', assumeNotNull(date_order))[1], '-',
                                    splitByString('-', assumeNotNull(date_order))[3], '-',
                                    splitByString('-', assumeNotNull(date_order))[2])) as date_order,

                toDateOrNull(concat(splitByString('-', assumeNotNull(planned_data_order))[1], '-',
                                    splitByString('-', assumeNotNull(planned_data_order))[3], '-',
                                    splitByString('-', assumeNotNull(planned_data_order))[2])) as planned_data_order,

                toDateOrNull(concat(splitByString('-', assumeNotNull(result_data_order))[1], '-',
                                    splitByString('-', assumeNotNull(result_data_order))[3], '-',
                                    splitByString('-', assumeNotNull(result_data_order))[2])) as result_data_order,
                carrier_name, customer_cluster, delivery_type, number_order, seller_ID, warehouse_cluster, timeslot_number
    FROM default.logistics_online
    WHERE order_status = 'Доставлен' 
        AND date_order IS NOT NULL 
        AND date_order < result_data_order
        AND date_order != '1970-01-01'  
        AND planned_data_order != '1970-01-01'
        AND result_data_order != '1970-01-01'
    ) AS L
LEFT JOIN -- отношение количества возвратов к заказам в день можно найти если в этот день заказы были, т е для объединения нам нужно использовать left join
    (
    SELECT DISTINCT number_order
    FROM default.returns_online
    ) AS R
ON L.number_order=R.number_order
GROUP BY carrier_name
ORDER BY carrier_name ASC
"""
Return_Rate_carrier_name = ph.read_clickhouse(query, connection=connection)
Return_Rate_carrier_name

,carrier_name,Return_Rate
0,3PL 1,5.90
1,3PL 2,5.11
2,3PL 3,6.14
3,3PL 4,6.67
4,Самодоставка,5.78


самая большая доля возвратов у 3PL 4 и на этого перевозчика приходится и самый маленький обьем продаж

разрезы по кластерам склада:

In [72]:
query = """

SELECT
    warehouse_cluster,
    ROUND(COUNT(CASE WHEN R.number_order !=0 THEN R.number_order END)/COUNT(L.number_order)*100, 2) AS Return_Rate 
FROM 
    (
    SELECT
        toDateOrNull(concat(splitByString('-', assumeNotNull(date_order))[1], '-',
                            splitByString('-', assumeNotNull(date_order))[3], '-',
                            splitByString('-', assumeNotNull(date_order))[2])) as date_order,

        toDateOrNull(concat(splitByString('-', assumeNotNull(planned_data_order))[1], '-',
                            splitByString('-', assumeNotNull(planned_data_order))[3], '-',
                            splitByString('-', assumeNotNull(planned_data_order))[2])) as planned_data_order,

        toDateOrNull(concat(splitByString('-', assumeNotNull(result_data_order))[1], '-',
                            splitByString('-', assumeNotNull(result_data_order))[3], '-',
                            splitByString('-', assumeNotNull(result_data_order))[2])) as result_data_order,
        carrier_name, customer_cluster, delivery_type, number_order, seller_ID, warehouse_cluster, timeslot_number
    FROM default.logistics_online
    WHERE order_status = 'Доставлен' 
        AND date_order IS NOT NULL 
        AND date_order < result_data_order
        AND date_order != '1970-01-01'  
        AND planned_data_order != '1970-01-01'
        AND result_data_order != '1970-01-01'
    ) AS L
LEFT JOIN -- отношение количества возвратов к заказам в день можно найти если в этот день заказы были, т е для объединения нам нужно использовать left join
    (
    SELECT DISTINCT number_order
    FROM default.returns_online
    ) AS R
ON L.number_order=R.number_order
GROUP BY warehouse_cluster
ORDER BY warehouse_cluster ASC
"""
Return_Rate_warehouse_cluster = ph.read_clickhouse(query, connection=connection)
Return_Rate_warehouse_cluster

,warehouse_cluster,Return_Rate
0,Кластер Екатеринбург,6.84
1,Кластер Казань,6.18
2,Кластер Калининград,4.60
3,Кластер Краснодар,5.93
4,Кластер Москва,6.05
5,Кластер Московская область,6.47
6,Кластер Новосибирск,3.91
7,Кластер Санкт-Петербург,4.05
8,Кластер Северо-Запад,3.25
9,Кластер Хабаровск,14.29


самая высокая доля возвратов у кластера склада в Хабаровске (возможно, из-за срывов обещанных сроков доставки), а кластеры Центр и Северо-Запада работают практически без возвратов

разрезы по кластерам клиента:

In [73]:
query = """

SELECT
    customer_cluster,
    ROUND(COUNT(CASE WHEN R.number_order !=0 THEN R.number_order END)/COUNT(L.number_order)*100, 2) AS Return_Rate 
FROM 
    (
    SELECT
        toDateOrNull(concat(splitByString('-', assumeNotNull(date_order))[1], '-',
                            splitByString('-', assumeNotNull(date_order))[3], '-',
                            splitByString('-', assumeNotNull(date_order))[2])) as date_order,

        toDateOrNull(concat(splitByString('-', assumeNotNull(planned_data_order))[1], '-',
                            splitByString('-', assumeNotNull(planned_data_order))[3], '-',
                            splitByString('-', assumeNotNull(planned_data_order))[2])) as planned_data_order,

        toDateOrNull(concat(splitByString('-', assumeNotNull(result_data_order))[1], '-',
                            splitByString('-', assumeNotNull(result_data_order))[3], '-',
                            splitByString('-', assumeNotNull(result_data_order))[2])) as result_data_order,
        carrier_name, customer_cluster, delivery_type, number_order, seller_ID, warehouse_cluster, timeslot_number
    FROM default.logistics_online
    WHERE order_status = 'Доставлен' 
        AND date_order IS NOT NULL 
        AND date_order < result_data_order
        AND date_order != '1970-01-01'  
        AND planned_data_order != '1970-01-01'
        AND result_data_order != '1970-01-01'
    ) AS L
LEFT JOIN -- отношение количества возвратов к заказам в день можно найти если в этот день заказы были, т е для объединения нам нужно использовать left join
    (
    SELECT DISTINCT number_order
    FROM default.returns_online
    ) AS R
ON L.number_order=R.number_order
GROUP BY customer_cluster
ORDER BY customer_cluster ASC
  
"""
Return_Rate_customer_cluster = ph.read_clickhouse(query, connection=connection)
Return_Rate_customer_cluster

,customer_cluster,Return_Rate
0,Кластер Екатеринбург,5.58
1,Кластер Казань,4.59
2,Кластер Калининград,21.05
3,Кластер Краснодар,4.22
4,Кластер Москва,6.04
5,Кластер Московская область,6.69
6,Кластер Новосибирск,6.25
7,Кластер Санкт-Петербург,5.52
8,Кластер Северо-Запад,1.80
9,Кластер Хабаровск,0.00


чаще всего возвращают заказы клиенты из Калининграда, нет возвратов среди клиентов Хабаровска и Якутии. Не совсем понятна ситуация в Хабаровске, в кластере склада самая высокая доля возвратов, а в кластере клиентов возвратов нет, нужно изучать данные более подробно:

In [81]:
query = """
SELECT
L.number_order AS orders, 
R.number_order AS return_orders,
carrier_name, customer_cluster, delivery_type, number_order, seller_ID, warehouse_cluster, timeslot_number
FROM 
    (
    SELECT
        toDateOrNull(concat(splitByString('-', assumeNotNull(date_order))[1], '-',
                            splitByString('-', assumeNotNull(date_order))[3], '-',
                            splitByString('-', assumeNotNull(date_order))[2])) as date_order,

        toDateOrNull(concat(splitByString('-', assumeNotNull(planned_data_order))[1], '-',
                            splitByString('-', assumeNotNull(planned_data_order))[3], '-',
                            splitByString('-', assumeNotNull(planned_data_order))[2])) as planned_data_order,

        toDateOrNull(concat(splitByString('-', assumeNotNull(result_data_order))[1], '-',
                            splitByString('-', assumeNotNull(result_data_order))[3], '-',
                            splitByString('-', assumeNotNull(result_data_order))[2])) as result_data_order,
        carrier_name, customer_cluster, delivery_type, number_order, seller_ID, warehouse_cluster, timeslot_number
    FROM default.logistics_online
    WHERE order_status = 'Доставлен' 
        AND date_order IS NOT NULL 
        AND date_order < result_data_order
        AND date_order != '1970-01-01'  
        AND planned_data_order != '1970-01-01'
        AND result_data_order != '1970-01-01'
    ) AS L
LEFT JOIN -- отношение количества возвратов к заказам в день можно найти если в этот день заказы были, т е для объединения нам нужно использовать left join
    (
    SELECT DISTINCT number_order
    FROM default.returns_online
    ) AS R
ON L.number_order=R.number_order
WHERE warehouse_cluster = 'Кластер Хабаровск'

"""
task5_clusters = ph.read_clickhouse(query, connection=connection)
task5_clusters

,orders,return_orders,carrier_name,customer_cluster,delivery_type,number_order,seller_ID,warehouse_cluster,timeslot_number
0,345970453,0,3PL 2,Кластер Новосибирск,Самовывоз,345970453,3392612,Кластер Хабаровск,1
1,347443552,0,3PL 2,Кластер Хабаровск,Самовывоз,347443552,3392612,Кластер Хабаровск,1
2,347504436,0,3PL 2,Кластер Хабаровск,Самовывоз,347504436,3392612,Кластер Хабаровск,1
3,3492142076,0,3PL 2,Кластер Хабаровск,Самовывоз,3492142076,3392612,Кластер Хабаровск,1
4,347511343,0,3PL 2,Кластер Хабаровск,Самовывоз,347511343,3392612,Кластер Хабаровск,1
5,327747059,0,3PL 3,Кластер Москва,Самовывоз,327747059,4830,Кластер Хабаровск,1
6,32021214114,32021214114,3PL 3,Кластер Московская область,Самовывоз,32021214114,4830,Кластер Хабаровск,1


т е из складского кластера Хабаровска в рассматриваемый период было сделано 7 отгрузок, одна из которых в Подмосковье, по этому заказу и был сделан возврат

### Задача 6.

Посчитать GMV D-R - метрика, отражающая совокупный объем доставленных заказов за вычетом возвратов, в рублях.

 - Необходимо видеть изменение метрики по дням (для всех дней, которые были раньше, чем дата расчета метрики);

 - Необходимы разрезы по перевозчикам, по типу доставки, по кластерам склада.

In [82]:
query = """
SELECT
    SUM(GMV_D_R) AS GMV_D_R
FROM 
    (
    SELECT
        toDateOrNull(concat(splitByString('-', assumeNotNull(date_order))[1], '-',
                            splitByString('-', assumeNotNull(date_order))[3], '-',
                            splitByString('-', assumeNotNull(date_order))[2])) as date_order,

        toDateOrNull(concat(splitByString('-', assumeNotNull(planned_data_order))[1], '-',
                            splitByString('-', assumeNotNull(planned_data_order))[3], '-',
                            splitByString('-', assumeNotNull(planned_data_order))[2])) as planned_data_order,

        toDateOrNull(concat(splitByString('-', assumeNotNull(result_data_order))[1], '-',
                            splitByString('-', assumeNotNull(result_data_order))[3], '-',
                            splitByString('-', assumeNotNull(result_data_order))[2])) as result_data_order,
        carrier_name, customer_cluster, delivery_type, number_order, seller_ID, warehouse_cluster, timeslot_number
    FROM default.logistics_online
    WHERE order_status = 'Доставлен' 
        AND date_order IS NOT NULL 
        AND date_order < result_data_order
        AND date_order != '1970-01-01'  
        AND planned_data_order != '1970-01-01'
        AND result_data_order != '1970-01-01'
    ) AS L1
JOIN 
    (
    SELECT 
        L.number_order,
        L.product_ID,
        L.product_price,
        L.amount,
        R.amount_return,
        L.amount*L.product_price AS sum_order,            
        R.amount_return*L.product_price AS sum_return,
        sum_order-sum_return AS GMV_D_R            
    FROM default.product_online AS L
    LEFT JOIN -- нам нужно добавить информацию по возвратам к таблице покупок, поэтому используем LEFT JOIN
           (
            SELECT DISTINCT -- в базе возвратов были найдены дубли, т е один и тот же product_ID из конкретного заказа мог возвращаться в разные дни, хотя в первоначальном заказе количество позиции было меньше
                number_order,
                product_ID,
                amount_return
            FROM default.returns_online
           ) AS R
    ON L.number_order=R.number_order 
    AND L.product_ID=R.product_ID
    ) AS R1
ON L1.number_order=R1.number_order

"""
task6 = ph.read_clickhouse(query, connection=connection)
task6

,GMV_D_R
0,78737343.9


изменение метрики по дням:

    SELECT
        date_order,
        SUM(GMV_D_R) AS GMV_D_R
    FROM 
        (
        SELECT
        toDateOrNull(concat(splitByString('-', assumeNotNull(date_order))[1], '-',
                            splitByString('-', assumeNotNull(date_order))[3], '-',
                            splitByString('-', assumeNotNull(date_order))[2])) as date_order,

        toDateOrNull(concat(splitByString('-', assumeNotNull(planned_data_order))[1], '-',
                            splitByString('-', assumeNotNull(planned_data_order))[3], '-',
                            splitByString('-', assumeNotNull(planned_data_order))[2])) as planned_data_order,

        toDateOrNull(concat(splitByString('-', assumeNotNull(result_data_order))[1], '-',
                            splitByString('-', assumeNotNull(result_data_order))[3], '-',
                            splitByString('-', assumeNotNull(result_data_order))[2])) as result_data_order,
            carrier_name, customer_cluster, delivery_type, number_order, seller_ID, warehouse_cluster, timeslot_number
        FROM default.logistics_online
        WHERE order_status = 'Доставлен' 
            AND date_order IS NOT NULL 
            AND date_order < result_data_order
            AND date_order != '1970-01-01'  
            AND planned_data_order != '1970-01-01'
            AND result_data_order != '1970-01-01'
            
            
            AND date_order < '' -- ВПИШИТЕ В КАВЫЧКИ ДАТУ РАСЧЕТА МЕТРИКИ В ФОРМАТЕ YYYY-MM-DD 
            
            
        ) AS L1
    JOIN 
        (
        SELECT 
            L.number_order,
            L.product_ID,
            L.product_price,
            L.amount,
            R.amount_return,
            L.amount*L.product_price AS sum_order,            
            R.amount_return*L.product_price AS sum_return,
            sum_order-sum_return AS GMV_D_R            
        FROM default.product_online AS L
        LEFT JOIN 
               (
                SELECT DISTINCT -- в базе возвратов были найдены дубли, т е один и тот же product_ID из конкретного заказа мог возвращаться в разные дни, хотя в первоначальном заказе количество позиции было меньше, чем суммарный показатель возвратов
                    number_order,
                    product_ID,
                    amount_return
                FROM default.returns_online
               ) AS R
        ON L.number_order=R.number_order 
        AND L.product_ID=R.product_ID
        ) AS R1
    ON L1.number_order=R1.number_order
    GROUP BY date_order
    ORDER By date_order ASC

In [83]:
GMV_D_R_by_days = pd.read_csv('GMV_D_R_by_days.csv')

In [84]:
GMV_D_R_by_days

,date_order,GMV_D_R
0,01/01/21,165253.86
1,02/01/21,212730.40
2,03/01/21,367444.84
3,04/01/21,350296.26
4,05/01/21,547738.82
...,...,...
68,10/03/21,817145.20
69,11/03/21,607281.99
70,12/03/21,264715.44
71,13/03/21,166059.53


разрезы по перевозчикам, по типу доставки, по кластерам склада:

разрезы по перевозчикам:

In [85]:
query = """
SELECT
    carrier_name,
    SUM(GMV_D_R) AS GMV_D_R
FROM 
    (
    SELECT
        toDateOrNull(concat(splitByString('-', assumeNotNull(date_order))[1], '-',
                            splitByString('-', assumeNotNull(date_order))[3], '-',
                            splitByString('-', assumeNotNull(date_order))[2])) as date_order,

        toDateOrNull(concat(splitByString('-', assumeNotNull(planned_data_order))[1], '-',
                            splitByString('-', assumeNotNull(planned_data_order))[3], '-',
                            splitByString('-', assumeNotNull(planned_data_order))[2])) as planned_data_order,

        toDateOrNull(concat(splitByString('-', assumeNotNull(result_data_order))[1], '-',
                            splitByString('-', assumeNotNull(result_data_order))[3], '-',
                            splitByString('-', assumeNotNull(result_data_order))[2])) as result_data_order,
        carrier_name, customer_cluster, delivery_type, number_order, seller_ID, warehouse_cluster, timeslot_number
    FROM default.logistics_online
    WHERE order_status = 'Доставлен' 
        AND date_order IS NOT NULL 
        AND date_order < result_data_order
        AND date_order != '1970-01-01'  
        AND planned_data_order != '1970-01-01'
        AND result_data_order != '1970-01-01'
    ) AS L1
JOIN 
    (
    SELECT 
        L.number_order,
        L.product_ID,
        L.product_price,
        L.amount,
        R.amount_return,
        L.amount*L.product_price AS sum_order,            
        R.amount_return*L.product_price AS sum_return,
        sum_order-sum_return AS GMV_D_R            
    FROM default.product_online AS L
    LEFT JOIN 
           (
            SELECT DISTINCT -- в базе возвратов были найдены дубли, т е один и тот же product_ID из конкретного заказа мог возвращаться в разные дни, хотя в первоначальном заказе количество позиции было меньше
                number_order,
                product_ID,
                amount_return
            FROM default.returns_online
           ) AS R
    ON L.number_order=R.number_order 
    AND L.product_ID=R.product_ID
    ) AS R1
ON L1.number_order=R1.number_order
GROUP BY carrier_name
ORDER By GMV_D_R DESC

"""
GMV_D_R_carrier_name = ph.read_clickhouse(query, connection=connection)
GMV_D_R_carrier_name

,carrier_name,GMV_D_R
0,3PL 1,25778072.51
1,3PL 2,22493589.59
2,Самодоставка,22088879.05
3,3PL 3,5585226.63
4,3PL 4,2791576.12


как и в метрике GMV Accepted на 3PL 1 приходится больше всего совокупного объема продаж

разрезы по типу доставки:

In [86]:
query = """
SELECT
    delivery_type,
    SUM(GMV_D_R) AS GMV_D_R
FROM 
    (
    SELECT
        toDateOrNull(concat(splitByString('-', assumeNotNull(date_order))[1], '-',
                            splitByString('-', assumeNotNull(date_order))[3], '-',
                            splitByString('-', assumeNotNull(date_order))[2])) as date_order,

        toDateOrNull(concat(splitByString('-', assumeNotNull(planned_data_order))[1], '-',
                            splitByString('-', assumeNotNull(planned_data_order))[3], '-',
                            splitByString('-', assumeNotNull(planned_data_order))[2])) as planned_data_order,

        toDateOrNull(concat(splitByString('-', assumeNotNull(result_data_order))[1], '-',
                            splitByString('-', assumeNotNull(result_data_order))[3], '-',
                            splitByString('-', assumeNotNull(result_data_order))[2])) as result_data_order,
        carrier_name, customer_cluster, delivery_type, number_order, seller_ID, warehouse_cluster, timeslot_number
    FROM default.logistics_online
    WHERE order_status = 'Доставлен' 
        AND date_order IS NOT NULL 
        AND date_order < result_data_order
        AND date_order != '1970-01-01'  
        AND planned_data_order != '1970-01-01'
        AND result_data_order != '1970-01-01'
    ) AS L1
JOIN 
    (
    SELECT 
        L.number_order,
        L.product_ID,
        L.product_price,
        L.amount,
        R.amount_return,
        L.amount*L.product_price AS sum_order,            
        R.amount_return*L.product_price AS sum_return,
        sum_order-sum_return AS GMV_D_R            
    FROM default.product_online AS L
    LEFT JOIN 
           (
            SELECT DISTINCT -- в базе возвратов были найдены дубли, т е один и тот же product_ID из конкретного заказа мог возвращаться в разные дни, хотя в первоначальном заказе количество позиции было меньше
                number_order,
                product_ID,
                amount_return
            FROM default.returns_online
           ) AS R
    ON L.number_order=R.number_order 
    AND L.product_ID=R.product_ID
    ) AS R1
ON L1.number_order=R1.number_order
GROUP BY delivery_type
ORDER By GMV_D_R DESC

"""
GMV_D_R_delivery_type = ph.read_clickhouse(query, connection=connection)
GMV_D_R_delivery_type

,delivery_type,GMV_D_R
0,Курьер,40804525.35
1,Самовывоз,37932818.55


разрезы по кластерам склада:

In [87]:
query = """
SELECT
    warehouse_cluster,
    SUM(GMV_D_R) AS GMV_D_R
FROM 
    (
    SELECT
        toDateOrNull(concat(splitByString('-', assumeNotNull(date_order))[1], '-',
                            splitByString('-', assumeNotNull(date_order))[3], '-',
                            splitByString('-', assumeNotNull(date_order))[2])) as date_order,

        toDateOrNull(concat(splitByString('-', assumeNotNull(planned_data_order))[1], '-',
                            splitByString('-', assumeNotNull(planned_data_order))[3], '-',
                            splitByString('-', assumeNotNull(planned_data_order))[2])) as planned_data_order,

        toDateOrNull(concat(splitByString('-', assumeNotNull(result_data_order))[1], '-',
                            splitByString('-', assumeNotNull(result_data_order))[3], '-',
                            splitByString('-', assumeNotNull(result_data_order))[2])) as result_data_order,
        carrier_name, customer_cluster, delivery_type, number_order, seller_ID, warehouse_cluster, timeslot_number
    FROM default.logistics_online
    WHERE order_status = 'Доставлен' 
        AND date_order IS NOT NULL 
        AND date_order < result_data_order
        AND date_order != '1970-01-01'  
        AND planned_data_order != '1970-01-01'
        AND result_data_order != '1970-01-01'
    ) AS L1
JOIN 
    (
    SELECT 
        L.number_order,
        L.product_ID,
        L.product_price,
        L.amount,
        R.amount_return,
        L.amount*L.product_price AS sum_order,            
        R.amount_return*L.product_price AS sum_return,
        sum_order-sum_return AS GMV_D_R            
    FROM default.product_online AS L
    LEFT JOIN 
           (
            SELECT DISTINCT -- в базе возвратов были найдены дубли, т е один и тот же product_ID из конкретного заказа мог возвращаться в разные дни, хотя в первоначальном заказе количество позиции было меньше
                number_order,
                product_ID,
                amount_return
            FROM default.returns_online
           ) AS R
    ON L.number_order=R.number_order 
    AND L.product_ID=R.product_ID
    ) AS R1
ON L1.number_order=R1.number_order
GROUP BY warehouse_cluster
ORDER By GMV_D_R DESC

"""
GMV_D_R_warehouse_cluster = ph.read_clickhouse(query, connection=connection)
GMV_D_R_warehouse_cluster

,warehouse_cluster,GMV_D_R
0,Кластер Москва,25231949.88
1,Кластер Московская область,12787876.73
2,Кластер Казань,7612094.82
3,Кластер Екатеринбург,7253831.98
4,Кластер Краснодар,6922070.42
5,Кластер Санкт-Петербург,6841138.33
6,Кластер Новосибирск,4628749.51
7,Кластер Центр,3379923.94
8,Кластер Северо-Запад,2047665.79
9,Кластер Калининград,1922571.55


как и в метрике GMV Accepted предсказуемо больше всего совокупного объема продаж приходится на кластер склада Москвы, далее идут Подмосковье и Казань, меньше всего дохода приносит кластер Хабаровска

См., пожалуйста, презентацию: https://redash.lab.karpov.courses/dashboards/2365-metrics_for_delivery

* в презентации метрика OnTime включает в себя заказы, доставленные как в плановую дату доставки, так и раньше

если ссылка не открывается, смотрите, пожалуйста, презентацию в PDF